# Initialization

In [2]:
import numpy as np
from matplotlib import pyplot as plt

# for p-value out of significance test
from scipy.stats import ttest_ind

# for image data handling
import os
from os.path import join, isfile, dirname
from PIL import Image

# for svm
from libsvm.svmutil import *

# Data Handling

### Uncompress compressed files

In [3]:
%%capture
!unzip -n ../data/images.zip -d data

### Custom functions

In [4]:
def genFromImage(imageDir, size=(8, 8)):
    dir = dirname(imageDir)
    dataFile = join(dir, "p4_data") + str(size) + ".npy"
    if isfile(dataFile):
        with open(dataFile, 'rb') as f:
            return np.load(f)
    
    labels = os.listdir(imageDir)
    image_data = [[] for _ in labels]
    for label in labels:
        dir = join(imageDir, label)
        files = os.listdir(dir)
        image_data[int(label)] = np.array([np.array(
            Image.open(join(dir, file)).convert("L").resize(size), dtype='uint8'
        ) for file in files])
        
    image_data = np.array(image_data)
    with open(dataFile, 'wb') as f:
        np.save(f, image_data)
    return image_data

# returns X, Y, X_test, Y_test and classStats
def trainTestSplit(data, train_ratio, func):
    n = data.shape[0]
    m = int(np.floor(data.shape[1] * train_ratio))
    classStats = {}
    x_train, y_train, x_test, y_test = [[[] for _ in range(n)] for _ in range(4)]
    for label in range(n):
        x_train[label], y_train[label], classStats[label] = func(label, data[label][:m], True)
        x_test[label], y_test[label] = func(label, data[label][m:])
    
    X, Y, X_test, Y_test = [x.reshape(-1, x.shape[-1]) for x in [np.array(x) for x in [x_train, y_train, x_test, y_test]]]
    return X, Y.flatten(), X_test, Y_test.flatten(), classStats

def imgToFeatures(label, data, stats=False):
    X = np.array([x.flatten() for x in data]) / 255
    Y = label * np.ones(data.shape[0])
    if stats:
        return X, Y, { "mean": np.mean(X, axis=0), "cov": np.cov(X.T), "prior": data.shape[0], "data": X }
    return X, Y

def classify(x, classStats, density):
    label = -1
    max = -99999
    sum = 0
    prob = []
    for key in classStats:
        mean = classStats[key]["mean"]
        cov = classStats[key]["cov"]
        prior = classStats[key]["prior"]
        weights = classStats[key]["weights"] if "weights" in classStats[key] else []
        value = np.log(prior) + density(x, mean, cov, weights)
        prob.append(value)
        sum += value
        if value > max:
            max, label = value, key
    return np.r_[[label], (np.array(prob) / sum)]

class metrics:
    def accuracy(predicted, actual):
        m = actual.size
        correctCount = sum([1 if int(predicted[i]) == int(actual[i]) else 0 for i in range(m)])
        return correctCount / m
    
    def confusionMatrix(predicted, actual, n = 5):
        cnf = np.zeros((n, n), dtype='uint')
        for i in range(actual.size):
            cnf[int(actual[i])][int(predicted[i])] += 1
        return cnf
    
    def f1Score(cnf):
        sum_predict = np.sum(cnf, axis=0)
        sum_actual  = np.sum(cnf, axis=1)
        f1 = np.zeros(cnf.shape[1])
        for i in range(f1.size):
            TP = cnf[i][i]
            FP, FN = sum_predict[i] - TP, sum_actual[i] - TP
            p, r = TP/(TP + FP + 1e-8), TP/(TP + FN + 1e-8)
            f1[i] = 2 * p * r / (p + r + 1e-8)
        return f1
    
    def print(X, Y, X_test, Y_test, classStats, density, result=True):
        n_labels = len(classStats)
        train = np.array([classify(x, classStats, density) for x in X])
        test = np.array([classify(x, classStats, density) for x in X_test])
        # train = classify(X, classStats, density)
        # test = classify(X, classStats, density)
        y_train, p_train = train.T[0], train.T[1:].T
        y_test, p_test = test.T[0], test.T[1:].T

        cnf_train = metrics.confusionMatrix(y_train, Y, n_labels)
        cnf_test = metrics.confusionMatrix(y_test, Y_test, n_labels)
        acc_train = metrics.accuracy(y_train, Y)
        acc_test = metrics.accuracy(y_test, Y_test)
        f1_train = metrics.f1Score(cnf_train)
        f1_test = metrics.f1Score(cnf_test)

        print("------------------ Train ---------------------")
        print("Classification Accuracy : ", acc_train)
        print("F1 Score                : ", f1_train)
        print("------------------ Test ----------------------")
        print("Classification Accuracy : ", acc_test)
        print("F1 Score                : ", f1_test)

        if result:
            return [acc_train, f1_train], [acc_test, f1_test]

### Data extraction

In [5]:
dataFolder = "../data"
imageDir = join(dataFolder, "images")
imageDataDir = join(dataFolder, "p4_data.csv")

p1 = { "testDir": dataFolder + "/p1_test.csv", "trainDir": dataFolder + "/p1_train.csv" } # regression
p2 = { "testDir": dataFolder + "/p2_test.csv", "trainDir": dataFolder + "/p2_train.csv" } # regression
p3 = { "testDir": dataFolder + "/p3_test.csv", "trainDir": dataFolder + "/p3_train.csv" } # classification
p4 = {}                                                                                   # classification
p5 = {}                                                                                   # classification

p1["test"] = np.genfromtxt(p1["testDir"], delimiter=',')
p1["train"] = np.genfromtxt(p1["trainDir"], delimiter=',')
p2["test"] = np.genfromtxt(p2["testDir"], delimiter=',')
p2["train"] = np.genfromtxt(p2["trainDir"], delimiter=',')
p3["test"] = np.genfromtxt(p3["testDir"], delimiter=',')
p3["train"] = np.genfromtxt(p3["trainDir"], delimiter=',')
p4["data"] = genFromImage(imageDir, (16, 16))
p5["data"] = np.genfromtxt(dataFolder + "/PCA_MNIST.csv", delimiter=',')[1:]

print("--------------------------- Data Shapes ------------------------------")
print("    (Regression) p1[train]:      ", p1["train"].shape, ", p1[test]: ", p1["test"].shape)
print("    (Regression) p2[train]:      ", p2["train"].shape, ", p2[test]: ", p2["test"].shape)
print("(Classification) p3[train]:     ", p3["train"].shape, ", p3[test]: ", p3["test"].shape)
print("(Classification)  p4[data]:", p4["data"].shape)
print("(Classification)  p5[data]:     ", p5["data"].shape)

--------------------------- Data Shapes ------------------------------
    (Regression) p1[train]:       (10000, 3) , p1[test]:  (5000, 3)
    (Regression) p2[train]:       (10000, 4) , p2[test]:  (5000, 4)
(Classification) p3[train]:      (60000, 11) , p3[test]:  (15000, 11)
(Classification)  p4[data]: (10, 6000, 16, 16)
(Classification)  p5[data]:      (60000, 11)


# P4 (Neural Networks, MLP)

- Construct a Multi-layer Perception (MLP) or a feed-forward neural network to work on the K-MNIST dataset. 
- Experiment with at least 3 settings of the number of hidden layers and Neurons. 
- Explicitly code the Error Backpropagation algorithm as a class and use it on MLPs with different architectures and loss functions (CE, squared error loss).
- For this part, you should only use Numpy. 

Report the accuracy and F1 scores with all the considered configurations.

**DATA:** `images.zip(p4["data"])`

In [21]:
a = np.arange(24).reshape(2,3,4)
b = 2 * a
b

array([[[ 0,  2,  4,  6],
        [ 8, 10, 12, 14],
        [16, 18, 20, 22]],

       [[24, 26, 28, 30],
        [32, 34, 36, 38],
        [40, 42, 44, 46]]])

In [17]:
a = [[1,2,3], [3,4,5]]
b = [2, 1]
# np.log(a[range(2), b])
[np.log(a[i][b[i]]) for i in range(len(b))]

[1.0986122886681098, 1.3862943611198906]

In [ ]:
# layers: array of number of neurons per layer including input and output
def initialize(layers):
    weights = []
    for i in range(len(layers) - 1):
        weights.append(np.random.rand(layers[i] + 1, layers[i+1]))
    return np.array(weights)

def forward_propagation(X, weights, layers):
    y, cache = X, [X]
    for i, l in enumerate(layers):
        y = np.c_[np.ones(len(y)), y] @ weights[i]
        cache.append(y)
        y = l[1](y)
    return y, cache

def backward_propagation(output, y, weights, layers, cache):
    delta = lossGradient(output, y)
    for i, l in reversed(list(enumerate(layers))):
        delta = np.multiply(weights[i].T @ delta, grad(l[1])(cache[i])) # not cache[i -1] as len(cache) = 1 + len(layers)
        gradients.append(cache[i] @ delta
    return gradients

def softmax(x):
    return np.exp(x) / np.exp(x).sum()

# Cross-entropy loss with softmax
def computeLoss(output, y):
    p = softmax(output)
    return np.sum([-np.log(p[i][j]) for i, j in enumerate(y)]) / len(y)

# Cross-entropy loss with softmax
def lossGradient(output, y):
    p = softmax(output)
    return np.array([p[i][j] - 1 for i, j in enumerate(y)]) / len(y)

def accuracy(p, y):
    pred = np.argmax(p, axis=1)
    return np.sum(np.double(np.abs(pred - y) < 0.5)) / len(y)

# layers: array of tuples of number of neurons per layer and activation function excluding input
# example: [(2, relu), (5, sigmoid)]
# alpha: learning rate
def train(X, Y, layers, epochs, alpha):
    labels = np.unique(Y)
    weights = initialize([X.shape[1], *[l[0] for l in layers]])
    loss_history = []
    accuracy_history = []
    
    for i in range(epochs):
        output, cache = forward_propagation(X, weights, layers)
        
        loss = computeLoss(output, Y)
        loss_history.append(loss)
        accuracy = accuracy(output, Y)
        accuracy_history.append(accuracy)
        
        gradients = backward_propagation(output, Y, weights, layers, cache)
        weights = weights - alpha * gradients
        
    return weights, loss_history, accuracy_history

In [ ]:
p3["train"].shape, p3["test"].shape

In [ ]:
classStats = {}
for row in p3["train"]:
    label = int(row[-1]) - 1
    if label in classStats:
        classStats[label].append(row[:-1])
    else:
        classStats[label] = [row[:-1]]

# classStats = [np.array(data) for data in classStats]
for i in range(len(classStats)):
    data = np.array(classStats[i])
    classStats[i] = { "mean": np.mean(data, axis=0), "cov": np.cov(data.T), "prior": data.shape[0], "data": data }

In [ ]:
def splitData(data):
    # X = np.array([normalize(col) for col in data.T[:-1]]).T
    X = data.T[:-1].T
    Y = data.T[-1].T.astype("int") - 1
    return X, Y

X, Y = splitData(p3["train"])
X_test, Y_test = splitData(p3["test"])

X.shape, Y.shape, X_test.shape, Y_test.shape

In [ ]:
p3["result"] = [[] for _ in range(5)]
p3["result"][0] = metrics.print(X, Y, X_test, Y_test, classStats, logNormal)

# P5 (Neural Networks, CNN)

- Construct a CNN for the K-MNIST dataset and code the back-propagation algorithm with weight sharing and local-receptive fields. 
- Experiment with 3 different architectures and report the accuracy.

**DATA:** `images.zip (p4[data])`

## Data handling

In [ ]:
p4["splitData"] = [trainTestSplit(p4["data"], r, imgToFeatures) for r in [0.2, 0.3, 0.5, 0.7, 0.9]]

## Naive Bayes

In [ ]:
p4["result"] = [[] for _ in range(5)]

### Test split -- 20:80

In [ ]:
p4["result"][0] = metrics.print(*p4["splitData"][0], naiveLogNormal)

### Test split -- 30:70

In [ ]:
p4["result"][0] = metrics.print(*p4["splitData"][1], naiveLogNormal)

### Test split -- 50:50

In [ ]:
p4["result"][0] = metrics.print(*p4["splitData"][2], naiveLogNormal)

### Test split -- 70:30

In [ ]:
p4["result"][0] = metrics.print(*p4["splitData"][3], naiveLogNormal)

### Test split -- 90:10

In [ ]:
p4["result"][0] = metrics.print(*p4["splitData"][4], naiveLogNormal)

## GMM

In [ ]:
def printGmm(data, number_of_guassians=2):
    classStatsGMM = {}
    for label in data[-1]:
        classStatsGMM[label] = { "prior": data[-1][label]["prior"] }
        classStatsGMM[label]["weights"], classStatsGMM[label]["mean"], classStatsGMM[label]["cov"] = em(data[-1][label]["data"], number_of_guassians, 50)

    metrics.print(*data[:-1], classStatsGMM, logGMM, result=False)

### Test split -- 20:80

In [ ]:
printGmm(p4["splitData"][0])

### Test split -- 30:70

In [ ]:
printGmm(p4["splitData"][1])

### Test split -- 50:50

In [ ]:
printGmm(p4["splitData"][2])

### Test split -- 70:30

In [ ]:
printGmm(p4["splitData"][3])

### Test split -- 90:10

In [ ]:
printGmm(p4["splitData"][4])

## Logistic Regression

In [ ]:
def logisticRegressor(data):
    X_train,y_train_orig , X_test, y_test_orig, classStats = data
    num_classes = 10
    num_samples = y_train_orig.shape[0]
    y_train = np.zeros((num_samples, num_classes))
    for i in range(num_samples):
        y_train[i, int(y_train_orig[i]) - 1] = 1

    # Define sigmoid function
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Define softmax function
    def softmax(x):
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    # Initialize weights and biases
    num_features = X_train.shape[1]
    W = np.random.randn(num_features, num_classes)
    b = np.random.randn(num_classes)

    # Set hyperparameters
    learning_rate = 0.1
    num_iterations = 1000
    epsilon = 1e-8

    # Train model using gradient descent
    prev_loss = float('inf')
    for i in range(num_iterations):
        # Forward propagation
        z = np.dot(X_train, W) + b
        y_pred = softmax(z)

        # Compute loss
        loss = -np.sum(y_train * np.log(y_pred + epsilon)) / num_samples

        # Backward propagation
        dz = y_pred - y_train
        dW = np.dot(X_train.T, dz) / num_samples
        db = np.sum(dz, axis=0) / num_samples

        # Update weights and biases
        W -= learning_rate * dW
        b -= learning_rate * db

        # Check stopping criterion
        if prev_loss - loss < epsilon:
            print('Stopping criterion met')
            break

        prev_loss = loss

    # Evaluate model on test set
    z = np.dot(X_test, W) + b
    y_pred = np.argmax(softmax(z), axis=1) + 1
    accuracy = np.sum(y_pred == y_test_orig) / y_test_orig.shape[0]
    print('Test accuracy:', accuracy)

    z_train = np.dot(X_train, W) + b
    y_train_pred = np.argmax(softmax(z_train), axis=1) + 1
    train_loss = -np.sum(y_train * np.log(softmax(z_train) + epsilon)) / num_samples
    train_error_rate = 1 - np.sum(y_train_pred == y_train_orig) / y_train_orig.shape[0]
    print('Training empirical risk:', train_loss)
    print('Training error rate:', train_error_rate)

    # Compute empirical risk on test data
    num_samples_test = y_test_orig.shape[0]
    y_test = np.zeros((num_samples_test, num_classes))
    for i in range(num_samples_test):
        y_test[i, int(y_test_orig[i]) - 1] = 1

    z_test = np.dot(X_test, W) + b
    test_loss = -np.sum(y_test * np.log(softmax(z_test) + epsilon)) / num_samples_test
    test_error_rate = 1 - np.sum(y_pred == y_test_orig) / y_test_orig.shape[0]
    print('Test empirical risk:', test_loss)
    print('Test error rate:', test_error_rate)

    num_classes = len(np.unique(y_test_orig))
    confusion_matrix = np.zeros((num_classes, num_classes))
    for i in range(len(y_test_orig)):
        true_class = int(y_test_orig[i] - 1)
        predicted_class = int(y_pred[i] - 1)
        confusion_matrix[true_class, predicted_class] += 1
    # print('Confusion matrix:')
    # print(confusion_matrix)


    num_classes = len(np.unique(y_test_orig))
    f1_scores = np.zeros(num_classes)
    for i in range(num_classes):
        true_positives = confusion_matrix[i, i]
        false_positives = np.sum(confusion_matrix[:, i]) - true_positives
        false_negatives = np.sum(confusion_matrix[i, :]) - true_positives
        precision = true_positives / (true_positives + false_positives + 1e-8)
        recall = true_positives / (true_positives + false_negatives + 1e-8)
        f1_scores[i] = 2 * precision * recall / (precision + recall + 1e-8)
    print('Class-wise F1 score:')
    print(f1_scores)

    # Choose two classes
    class_1 = 1
    class_2 = 2
    # Get predicted probabilities for the two classes
    y_class_1 = y_pred == class_1
    y_class_2 = y_pred == class_2
    y_prob_1 = softmax(z)[:, class_1 - 1]
    y_prob_2 = softmax(z)[:, class_2 - 1]

    # Compute true positive rate and false positive rate for both classes
    num_thresholds = 100
    tpr_class_1 = np.zeros(num_thresholds)
    fpr_class_1 = np.zeros(num_thresholds)
    tpr_class_2 = np.zeros(num_thresholds)
    fpr_class_2 = np.zeros(num_thresholds)

    for i in range(num_thresholds):
        threshold = i / (num_thresholds - 1)
        tp_class_1 = np.sum((y_prob_1 >= threshold) & (y_class_1 == True))
        fn_class_1 = np.sum((y_prob_1 < threshold) & (y_class_1 == True))
        tn_class_1 = np.sum((y_prob_2 < threshold) & (y_class_2 == True))
        fp_class_1 = np.sum((y_prob_2 >= threshold) & (y_class_2 == False))
        tpr_class_1[i] = tp_class_1 / (tp_class_1 + fn_class_1 + 1e-8)
        fpr_class_1[i] = fp_class_1 / (fp_class_1 + tn_class_1 + 1e-8)

        tp_class_2 = np.sum((y_prob_2 >= threshold) & (y_class_2 == True))
        fn_class_2 = np.sum((y_prob_2 < threshold) & (y_class_2 == True))
        tn_class_2 = np.sum((y_prob_1 < threshold) & (y_class_1 == True))
        fp_class_2 = np.sum((y_prob_1 >= threshold) & (y_class_1 == False))
        tpr_class_2[i] = tp_class_2 / (tp_class_2 + fn_class_2 + 1e-8)
        fpr_class_2[i] = fp_class_2 / (fp_class_2 + tn_class_2 + 1e-8)

    # Plot RoC curves and confusion matrix
    fig, ax = plt.subplots(2, 2, figsize=(12, 12))
    ax[0, 0].matshow(confusion_matrix, cmap='GnBu')
    ax[0, 0].set_xlabel("Predicted")
    ax[0, 0].set_ylabel("Actual")
    ax[0, 0].set_title("Confusion Matrix")
    for (x, y), value in np.ndenumerate(confusion_matrix):
        ax[0, 0].text(x, y, f"{value: .0f}", va="center", ha="center")

    ax[0, 1].plot(fpr_class_1, tpr_class_1, marker='x')
    ax[0, 1].set_xlabel("False positive rate")
    ax[0, 1].set_ylabel("True positive rate")                     
    ax[0, 1].set_title("ROC curve for class {}".format(class_1))

    ax[1, 0].plot(fpr_class_2, tpr_class_2, marker='x')
    ax[1, 0].set_xlabel("False positive rate")
    ax[1, 0].set_ylabel("True Positive rate")
    ax[1, 0].set_title("ROC curve for class {}".format(class_2))

    ax[1, 1].plot(fpr_class_1, tpr_class_1, marker='x', label="Class {}".format(class_1))
    ax[1, 1].plot(fpr_class_2, tpr_class_2, marker='o', label="Class {}".format(class_2))
    ax[1, 1].set_xlabel("False positive rate")
    ax[1, 1].set_ylabel("True positive rate")
    ax[1, 1].set_title("ROC curve for classes {} and {}".format(class_1, class_2))
    ax[1, 1].legend()

    fig.tight_layout()
    plt.show()

### Test split -- 20:80

In [ ]:
logisticRegressor(p4["splitData"][0])

### Test split -- 30:70

In [ ]:
logisticRegressor(p4["splitData"][1])

### Test split -- 50:50

In [ ]:
logisticRegressor(p4["splitData"][2])

### Test split -- 70:30

In [ ]:
logisticRegressor(p4["splitData"][3])

### Test split -- 90:10

In [ ]:
logisticRegressor(p4["splitData"][4])

# P7 (Neural Networks, MLP)

Train an MLP on the PCA counterpart of the KMINST dataset and report your observations

**DATA:** `p5[data]`

## Data handling

In [ ]:
def stats(label, data, stats=False):
    X = data
    Y = label * np.ones(data.shape[0])
    if stats:
        return X, Y, { "mean": np.mean(X, axis=0), "cov": np.cov(X.T), "prior": data.shape[0], "data": X }
    return X, Y

classWiseData = [[] for _ in range(10)]
for row in p5["data"]:
    label = int(row[0])
    classWiseData[label].append(row[1:])
    
p5["splitData"] = [trainTestSplit(np.array(classWiseData), r, stats) for r in [0.2, 0.3, 0.5, 0.7, 0.9]]

## Naive Bayes

In [ ]:
p5["result"] = [[] for _ in range(5)]

### Test split -- 20:80

In [ ]:
p5["result"][0] = metrics.print(*p5["splitData"][0], naiveLogNormal)

### Test split -- 30:70

In [ ]:
p5["result"][0] = metrics.print(*p5["splitData"][1], naiveLogNormal)

### Test split -- 50:50

In [ ]:
p5["result"][0] = metrics.print(*p5["splitData"][2], naiveLogNormal)

### Test split -- 70:30

In [ ]:
p5["result"][0] = metrics.print(*p5["splitData"][3], naiveLogNormal)

### Test split -- 90:10

In [ ]:
p5["result"][0] = metrics.print(*p5["splitData"][4], naiveLogNormal)

## GMM

### Test split -- 20:80

In [ ]:
printGmm(p5["splitData"][0])

### Test split -- 30:70

In [ ]:
printGmm(p5["splitData"][1])

### Test split -- 50:50

In [ ]:
printGmm(p5["splitData"][2])

### Test split -- 70:30

In [ ]:
printGmm(p5["splitData"][3])

### Test split -- 90:10

In [ ]:
printGmm(p5["splitData"][4])

## Logistic Regression

In [ ]:
def logisticRegressor(data):
    X_train,y_train_orig , X_test, y_test_orig, classStats = data
    num_classes = 10
    num_samples = y_train_orig.shape[0]
    y_train = np.zeros((num_samples, num_classes))
    for i in range(num_samples):
        y_train[i, int(y_train_orig[i]) - 1] = 1

    # Define sigmoid function
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Define softmax function
    def softmax(x):
        # subtract the maximum value from x to avoid overflow
        x -= np.max(x, axis=1, keepdims=True)
        exp_x = np.exp(x)
        # divide by the sum of the exponential values along axis 1
        return exp_x / np.sum(exp_x, axis=1, keepdims=True, where=np.isfinite(exp_x))

    # Initialize weights and biases
    num_features = X_train.shape[1]
    W = np.random.randn(num_features, num_classes)
    b = np.random.randn(num_classes)

    # Set hyperparameters
    learning_rate = 0.1
    num_iterations = 1000
    epsilon = 1e-8

    # Train model using gradient descent
    prev_loss = float('inf')
    for i in range(num_iterations):
        # Forward propagation
        z = np.dot(X_train, W) + b
        y_pred = softmax(z)

        # Compute loss
        loss = -np.sum(y_train * np.log(y_pred + epsilon)) / num_samples

        # Backward propagation
        dz = y_pred - y_train
        dW = np.dot(X_train.T, dz) / num_samples
        db = np.sum(dz, axis=0) / num_samples

        # Update weights and biases
        W -= learning_rate * dW
        b -= learning_rate * db

        # Check stopping criterion
        if prev_loss - loss < epsilon:
            print('Stopping criterion met')
            break

        prev_loss = loss

    # Evaluate model on test set
    z = np.dot(X_test, W) + b
    y_pred = np.argmax(softmax(z), axis=1) + 1
    accuracy = np.sum(y_pred == y_test_orig) / y_test_orig.shape[0]
    print('Test accuracy:', accuracy)

    z_train = np.dot(X_train, W) + b
    y_train_pred = np.argmax(softmax(z_train), axis=1) + 1
    train_loss = -np.sum(y_train * np.log(softmax(z_train) + epsilon)) / num_samples
    train_error_rate = 1 - np.sum(y_train_pred == y_train_orig) / y_train_orig.shape[0]
    print('Training empirical risk:', train_loss)
    print('Training error rate:', train_error_rate)

    # Compute empirical risk on test data
    num_samples_test = y_test_orig.shape[0]
    y_test = np.zeros((num_samples_test, num_classes))
    for i in range(num_samples_test):
        y_test[i, int(y_test_orig[i]) - 1] = 1

    z_test = np.dot(X_test, W) + b
    test_loss = -np.sum(y_test * np.log(softmax(z_test) + epsilon)) / num_samples_test
    test_error_rate = 1 - np.sum(y_pred == y_test_orig) / y_test_orig.shape[0]
    print('Test empirical risk:', test_loss)
    print('Test error rate:', test_error_rate)

    num_classes = len(np.unique(y_test_orig))
    confusion_matrix = np.zeros((num_classes, num_classes))
    for i in range(len(y_test_orig)):
        true_class = int(y_test_orig[i] - 1)
        predicted_class = int(y_pred[i] - 1)
        confusion_matrix[true_class, predicted_class] += 1
    print('Confusion matrix:')
    print(confusion_matrix)

    num_classes = len(np.unique(y_test_orig))
    f1_scores = np.zeros(num_classes)
    for i in range(num_classes):
        true_positives = confusion_matrix[i, i]
        false_positives = np.sum(confusion_matrix[:, i]) - true_positives
        false_negatives = np.sum(confusion_matrix[i, :]) - true_positives
        precision = true_positives / (true_positives + false_positives + 1e-8)
        recall = true_positives / (true_positives + false_negatives + 1e-8)
        f1_scores[i] = 2 * precision * recall / (precision + recall + 1e-8)
    print('Class-wise F1 score:')
    print(f1_scores)





 # Choose two classes
    class_1 = 1
    class_2 = 2

    # Get predicted probabilities for the two classes
    y_class_1 = y_test_orig == class_1
    y_class_2 = y_test_orig == class_2
    y_prob_1 = softmax(z_test)[:, class_1 - 1]
    y_prob_2 = softmax(z_test)[:, class_2 - 1]

    # Compute true positive rate and false positive rate for both classes
    num_thresholds = 100
    tpr_class_1 = np.zeros(num_thresholds)
    fpr_class_1 = np.zeros(num_thresholds)
    tpr_class_2 = np.zeros(num_thresholds)
    fpr_class_2 = np.zeros(num_thresholds)

    for i in range(num_thresholds):
        threshold = i / (num_thresholds - 1)
        tp_class_1 = np.sum(y_class_1 & (y_prob_1 > threshold))
        fp_class_1 = np.sum(~y_class_1 & (y_prob_1 > threshold))
        tn_class_1 = np.sum(~y_class_1 & (y_prob_1 <= threshold))
        fn_class_1 = np.sum(y_class_1 & (y_prob_1 <= threshold))
        tpr_class_1[i] = tp_class_1 / (tp_class_1 + fn_class_1)
        fpr_class_1[i] = fp_class_1 / (fp_class_1 + tn_class_1)

        tp_class_2 = np.sum(y_class_2 & (y_prob_2 > threshold))
        fp_class_2 = np.sum(~y_class_2 & (y_prob_2 > threshold))
        tn_class_2 = np.sum(~y_class_2 & (y_prob_2 <= threshold))
        fn_class_2 = np.sum(y_class_2 & (y_prob_2 <= threshold))
        tpr_class_2[i] = tp_class_2 / (tp_class_2 + fn_class_2)
        fpr_class_2[i] = fp_class_2 / (fp_class_2 + tn_class_2)
        
    
    # Plot RoC curves and confusion matrix
    fig, ax = plt.subplots(2, 2, figsize=(12, 12))
    ax[0, 0].matshow(confusion_matrix, cmap='GnBu')
    ax[0, 0].set_xlabel("Predicted")
    ax[0, 0].set_ylabel("Actual")
    ax[0, 0].set_title("Confusion Matrix")
    for (x, y), value in np.ndenumerate(confusion_matrix):
        ax[0, 0].text(x, y, f"{value: .0f}", va="center", ha="center")


    ax[0, 1].plot(fpr_class_1, tpr_class_1, marker='x')
    ax[0, 1].set_xlabel("False positive rate")
    ax[0, 1].set_ylabel("True positive rate")                     
    ax[0, 1].set_title("ROC curve for class {}".format(class_1))

    ax[1, 0].plot(fpr_class_2, tpr_class_2, marker='x')
    ax[1, 0].set_xlabel("False positive rate")
    ax[1, 0].set_ylabel("True Positive rate")
    ax[1, 0].set_title("ROC curve for class {}".format(class_2))

    ax[1, 1].plot(fpr_class_1, tpr_class_1, marker='x', label="Class {}".format(class_1))
    ax[1, 1].plot(fpr_class_2, tpr_class_2, marker='o', label="Class {}".format(class_2))
    ax[1, 1].set_xlabel("False positive rate")
    ax[1, 1].set_ylabel("True positive rate")
    ax[1, 1].set_title("ROC curve for classes {} and {}".format(class_1, class_2))
    ax[1, 1].legend()

    fig.tight_layout()
    plt.show()





### Test split -- 20:80

In [ ]:
logisticRegressor(p5["splitData"][0])

### Test split -- 30:70

In [ ]:
logisticRegressor(p5["splitData"][1])

### Test split -- 50:50

In [ ]:
logisticRegressor(p5["splitData"][2])

### Test split -- 70:30

In [ ]:
logisticRegressor(p5["splitData"][3])

### Test split -- 90:10

In [ ]:
logisticRegressor(p5["splitData"][4])